In [1]:
from openai import OpenAI
client = OpenAI()

In [4]:
import pandas as pd
df = pd.read_csv("../data/kaggle_restaurant_data.csv")
df.head()

/var/folders/zs/_4r576nj1rn83fbwgxx08g1w0000gn/T/ipykernel_29921/771314455.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/kaggle_restaurant_data.csv")


,id,restaurant_name,score,ratings,restaurant_type,full_address,menu_category,menu_name,menu_item_description,price
0,5.0,Nelson Brothers Cafe (17th St N),4.7,22.0,"Breakfast and Brunch, Burgers, Sandwiches","314 17th St N, Birmingham, AL, 35203",Picked for you,Pork Chop Rice with Gravy Plate,NaN,7.00
1,5.0,Nelson Brothers Cafe (17th St N),4.7,22.0,"Breakfast and Brunch, Burgers, Sandwiches","314 17th St N, Birmingham, AL, 35203",Picked for you,Full Sausage (2 pcs) with 2 Eggs,2 pieces.,7.25
2,5.0,Nelson Brothers Cafe (17th St N),4.7,22.0,"Breakfast and Brunch, Burgers, Sandwiches","314 17th St N, Birmingham, AL, 35203",Picked for you,Bacon and Egg with Cheese Breakfast Sandwich,NaN,3.50
3,5.0,Nelson Brothers Cafe (17th St N),4.7,22.0,"Breakfast and Brunch, Burgers, Sandwiches","314 17th St N, Birmingham, AL, 35203",Picked for you,Double Cheese Burger,Grilled or fried patty with cheese on a bun.,3.25
4,5.0,Nelson Brothers Cafe (17th St N),4.7,22.0,"Breakfast and Brunch, Burgers, Sandwiches","314 17th St N, Birmingham, AL, 35203",Picked for you,Full Bacon (3 pcs) with 2 Eggs,3 pieces.,7.25


In [9]:
class OpenAIConnector:
    def __init__(self, token_path="../credentials/open_ai_token.txt"):
        self.token = self._load_token(token_path)
        self.client = OpenAI(api_key=self.token)

    @staticmethod
    def _load_token(token_path):
        try:
            with open(token_path, "r") as f:
                token = f.read().strip()
                if not token:
                    raise ValueError("Token file is empty.")
                return token
        except FileNotFoundError:
            raise FileNotFoundError(f"Token file not found at {token_path}")
        except Exception as e:
            raise RuntimeError(f"Error reading token: {e}")

    def make_request(self, model="o4-mini", input_text="Convert 1 kg to lbs"):
        response = self.client.responses.create(
            model=model,
            input=input_text
        )
        return response.output_text
    def get_usage_summary(self):
        headers = {
            "Authorization": f"Bearer {self.token}"
        }

        now = datetime.now(timezone.utc)
        start_date = now.replace(day=1).strftime("%Y-%m-%d")
        end_date = now.strftime("%Y-%m-%d")

        # 1. Get usage data
        usage_url = f"https://api.openai.com/v1/dashboard/billing/usage?start_date={start_date}&end_date={end_date}"
        usage_resp = requests.get(usage_url, headers=headers)
        usage_data = usage_resp.json()
        used_usd = usage_data.get("total_usage", 0) / 100.0  # from cents to dollars

        # 2. Get allowance (subscription limit)
        limits_url = "https://api.openai.com/v1/dashboard/billing/subscription"
        limits_resp = requests.get(limits_url, headers=headers)
        limits_data = limits_resp.json()
        hard_limit = limits_data.get("hard_limit_usd", 0)
        soft_limit = limits_data.get("soft_limit_usd", 0)

        remaining = hard_limit - used_usd

        return {
            "used_usd": round(used_usd, 2),
            "soft_limit_usd": round(soft_limit, 2),
            "hard_limit_usd": round(hard_limit, 2),
            "remaining_usd": round(remaining, 2)
        }

In [5]:
df_20 = df[["restaurant_name", "restaurant_type", "menu_name"]].head(20)
df_20

,restaurant_name,restaurant_type,menu_name
0,Nelson Brothers Cafe (17th St N),"Breakfast and Brunch, Burgers, Sandwiches",Pork Chop Rice with Gravy Plate
1,Nelson Brothers Cafe (17th St N),"Breakfast and Brunch, Burgers, Sandwiches",Full Sausage (2 pcs) with 2 Eggs
2,Nelson Brothers Cafe (17th St N),"Breakfast and Brunch, Burgers, Sandwiches",Bacon and Egg with Cheese Breakfast Sandwich
3,Nelson Brothers Cafe (17th St N),"Breakfast and Brunch, Burgers, Sandwiches",Double Cheese Burger
4,Nelson Brothers Cafe (17th St N),"Breakfast and Brunch, Burgers, Sandwiches",Full Bacon (3 pcs) with 2 Eggs
5,Nelson Brothers Cafe (17th St N),"Breakfast and Brunch, Burgers, Sandwiches",Full Ham with 2 Eggs
6,Nelson Brothers Cafe (17th St N),"Breakfast and Brunch, Burgers, Sandwiches",1/2 Ham with 1 Egg
7,Nelson Brothers Cafe (17th St N),"Breakfast and Brunch, Burgers, Sandwiches",Full Bacon (3 pcs) with 2 Eggs
8,Nelson Brothers Cafe (17th St N),"Breakfast and Brunch, Burgers, Sandwiches",1/2 Bacon (2 pcs) with 1 Egg
9,Nelson Brothers Cafe (17th St N),"Breakfast and Brunch, Burgers, Sandwiches",Full Sausage (2 pcs) with 2 Eggs


In [6]:
len(df_20)

20

### Test Prompt
You are a data extractor. Extract from input as:  
    restaurant_name = Domino's  
    restaurant_type = pizza  
    menu_name = pepperoni pizza  
For each dish, extract these:  
    Dish cuisine (e.g. "Pizza restaurant")  
    Dish base (main ingredient, e.g. "Pizza")  
    Dish flavor (flavor, e.g. "Pepperoni")  

If the restaurant's category is missing or unclear, infer the cuisine from the dish name. 

Examples:  
Input: 
```text
restaurant_name = Summer restaurant
restaurant_type = Chinese, Comfort Food
menu_name = Beef Broccoli
```
Output: 
```json
[
  {
    "dish_cuisine": "Chinese restaurant",
    "dish_base": "Beef",
    "dish_flavor": "Broccoli"
  }
]
```
Process the following inputs: 
restaurant_name = 
restaurant_type = 
menu_name = 

In [13]:
import math
import pandas as pd

# Assume BATCH_SIZE = 10
def prepare_input_batches(df: pd.DataFrame, batch_size: int) -> list[str]:
    
    # 1) System instruction
    system_inst = (
        "You are a data extractor. Extract from input as:\n"
        "    restaurant_name = Domino's\n"
        "    restaurant_type = pizza\n"
        "    menu_name = pepperoni pizza\n"
        "For each dish, extract these:\n"
        "    Dish cuisine (e.g. \"Pizza restaurant\")\n"
        "    Dish base (main ingredient, e.g. \"Pizza\")\n"
        "    Dish flavor (flavor, e.g. \"Pepperoni\")\n\n"
        "If the restaurant's category is missing or unclear, infer the cuisine from the dish name.\n\n"
        "Examples:\n"
        "Input:\n"
        "```text\n"
        "restaurant_name = Summer restaurant\n"
        "restaurant_type = Chinese, Comfort Food\n"
        "menu_name = Beef Broccoli\n"
        "```\n"
        "Output:\n"
        "```json\n"
        "[\n"
        "  {\n"
        "    \"dish_cuisine\": \"Chinese restaurant\",\n"
        "    \"dish_base\": \"Beef\",\n"
        "    \"dish_flavor\": \"Broccoli\"\n"
        "  }\n"
        "]\n"
        "```\n\n"
        "Process the following inputs:\n"
    )
    
    # 2) Template (user part)
    tmpl = (
        "restaurant_name = {restaurant_name}\n"
        "restaurant_type = {restaurant_type}\n"
        "menu_name = {menu_name}\n\n"
    )
    
    # 3) Generate all pieces of prompts
    pieces = [
        tmpl.format(
            menu_name=row["menu_name"],
            restaurant_name=row["restaurant_name"],
            restaurant_type=row["restaurant_type"]
        )
        for _, row in df.iterrows()
    ]
    
    # 4) Slice and generate based on batch size
    n_batches = math.ceil(len(pieces) / batch_size)
    batches = []
    for i in range(n_batches):
        start, end = i * batch_size, (i + 1) * batch_size
        user_block = "".join(pieces[start:end])
        batches.append(system_inst + user_block)
    return batches

# test 
batches = prepare_input_batches(df_20, batch_size=10)
prompt_text = ''
for inp in batches:
    prompt_text += inp + '\n'
print(prompt_text)

You are a data extractor. Extract from input as:
    restaurant_name = Domino's
    restaurant_type = pizza
    menu_name = pepperoni pizza
For each dish, extract these:
    Dish cuisine (e.g. "Pizza restaurant")
    Dish base (main ingredient, e.g. "Pizza")
    Dish flavor (flavor, e.g. "Pepperoni")

If the restaurant's category is missing or unclear, infer the cuisine from the dish name.

Examples:
Input:
```text
restaurant_name = Summer restaurant
restaurant_type = Chinese, Comfort Food
menu_name = Beef Broccoli
```
Output:
```json
[
  {
    "dish_cuisine": "Chinese restaurant",
    "dish_base": "Beef",
    "dish_flavor": "Broccoli"
  }
]
```

Process the following inputs:
restaurant_name = Nelson Brothers Cafe (17th St N)
restaurant_type = Breakfast and Brunch, Burgers, Sandwiches
menu_name = Pork Chop Rice with Gravy Plate

restaurant_name = Nelson Brothers Cafe (17th St N)
restaurant_type = Breakfast and Brunch, Burgers, Sandwiches
menu_name = Full Sausage (2 pcs) with 2 Eggs

re

In [14]:
connector = OpenAIConnector()
print(connector.make_request(input_text=prompt_text))

[  
  {  
    "dish_cuisine": "Breakfast and Brunch restaurant",  
    "dish_base": "Sausage",  
    "dish_flavor": "Egg"  
  },  
  {  
    "dish_cuisine": "Breakfast and Brunch restaurant",  
    "dish_base": "Salmon",  
    "dish_flavor": "Egg"  
  },  
  {  
    "dish_cuisine": "Breakfast and Brunch restaurant",  
    "dish_base": "Salmon",  
    "dish_flavor": "Egg"  
  },  
  {  
    "dish_cuisine": "Breakfast and Brunch restaurant",  
    "dish_base": "Frank",  
    "dish_flavor": "Egg"  
  },  
  {  
    "dish_cuisine": "Breakfast and Brunch restaurant",  
    "dish_base": "Frank",  
    "dish_flavor": "Egg"  
  },  
  {  
    "dish_cuisine": "Breakfast and Brunch restaurant",  
    "dish_base": "Grits",  
    "dish_flavor": "Fish"  
  },  
  {  
    "dish_cuisine": "Breakfast and Brunch restaurant",  
    "dish_base": "Grits",  
    "dish_flavor": "Egg"  
  },  
  {  
    "dish_cuisine": "Breakfast and Brunch restaurant",  
    "dish_base": "Grits",  
    "dish_flavor": ""  
 